<div align="right">  
    Grupo 13  
    
    André Neves da Costa - A95869 
    Filipe José Silva Castro - A96156 
</div>

## Variáveis de Input:

+ *S* $\rightarrow$ Número de **salas**
+ *T* $\rightarrow$ Número de **slots**
+ *P* $\rightarrow$ Número de **projetos**
+ *C* $\rightarrow$ Número de **colaboradores**
+ *projetos* $\rightarrow$ Dicionário cujas keys são os **projetos** e os values são um tuplo com a **lista de colaboradores**, **líder do projeto** e **número mínimo de reuniões** desse projeto
+ *disp* $\rightarrow$ Lista de listas que representa a **disponibilidade de cada colaborador**, cada posição de disp está associada ao colaborador com o mesmo número (lista no posição 0 de disp representa a disponibilidade do colaborador 0

## Variáveis auxiliares:

+ $X_{s,t,p,c}\rightarrow$ Representa a participação do colaborador **c** na reunião do projeto **p** no slot **t** e na sala **s**,
$s\in[1..S],t\in[1..T],p\in[1..P],c\in[1..C]$

## Condições:

1. Em cada sala e slot pode ter no maximo uma reunião
2. O lider de cada projeto tem que participar em todas as reuniões
3. Em cada reunião têm que participar no minimo 50% dos seus colaboradores
4. Cada projeto tem um numero minimo de reunioes semanais
5. Os colaboradores só conseguem participar em reuniões em slots que têm disponibilidade
6. Os colaboradores não podem ter mais do que uma reunião no mesmo slot
7. Cada colaborador só pode participar em reuniões de projetos em que está incluido
8. Cada projeto só pode ter 1 ou 0 reuniões em cada slot

## Critérios de otimização:

1. Maximizar o número de reuniões efetivamente realizadas
2. Minimizar o número médio de reuniões por participante

## Implementação em python:

In [ ]:
from ortools.linear_solver import pywraplp

horario = pywraplp.Solver.CreateSolver('SCIP')

S, T, P, C = 2, 4, 2, 6

#projetos = {projeto:([colaboradores],lider,nº minimo reunioes)}
projetos = {0:([0,1,2],1,2), 1:([3,4,5],3,2)}

disp = [[1,1,1,1],[1,1,1,1],[1,1,0,1],[1,1,1,1],[1,1,1,1],[1,0,1,1]]



'''

S, T, P, C = 2,5,4,10

projetos = {0:([0,1,2],1,2), 1:([3,4,5],3,2),2:([1,4,8,9],9,1),3:([3,6,7,8],6,2)}

disp = [[1,1,1,1,1],[1,1,1,1,0],[1,1,0,1,1],[0,1,1,1,1],[1,1,1,1,0],[1,0,1,1,1],[0,1,1,1,1],[1,1,0,1,1],[0,1,1,1,0],[1,1,1,0,1]]



S, T, P, C = 4,6,4,12

projetos = {0:([0,1,2,4,9,11],1,2), 1:([2,4,7,9],3,2),2:([1,2,7,9,10],9,3),3:([3,6,7,8,11],6,2)}

disp = []
for i in range(12):
    disp.append([0,0,0,0,0,0])
'''

In [41]:
X = {}
for s in range(S):
    for t in range(T):
        for p in range(P):
            for c in range(C):
                X[s,t,p,c] = horario.BoolVar(f'X[{s}][{t}][{p}][{c}]')

1. Em cada sala e slot pode ter no maximo uma reunião 
$$\forall_{s<S} \forall_{t<T}, \sum_{p<P} X_{s,t,p,projetos[p][1]} \le 1$$

In [42]:
for s in range(S):
    for t in range(T):
        horario.Add(sum([X[s,t,p,projetos[p][1]] for p in range(P)]) <= 1)

2. O lider de cada projeto tem que participar em todas as reuniões$$ \forall_{t<T}\forall_{p<P}\forall_{s<S}\forall_{c<C},X_{s,t,p,c} \le X_{s,t,p,projetos[p][1]} $$

In [43]:
for t in range(T):
    for p in range(P):
        for s in range(S):
            for c in range(C):
                horario.Add(X[s,t,p,c] <= X[s,t,p,projetos[p][1]])

3. Em cada reunião têm que participar no minimo 50% dos seus colaboradores$$ \forall_{t<T}\forall_{p<P}\forall_{s<S},\sum_{c<C} X_{s,t,p,c} \ge len(projetos[p][0])/2 \times X_{s,t,p,projetos[p][1]}$$

In [44]:
for t in range(T):
    for p in range(P):
        for s in range(S):
            horario.Add(sum([X[s,t,p,c] for c in range(C)]) >= (len(projetos[p][0])/2)*X[s,t,p,projetos[p][1]])

4. Cada projeto tem um numero minimo de reunioes semanais$$ \forall_{p<P},\sum_{t<T,s<S} X_{s,t,p,lider[p]} \ge projetos[p][2] $$

In [45]:
for p in range(P):
        horario.Add(sum([X[s,t,p,projetos[p][1]] for t in range(T) for s in range(S)]) >= projetos[p][2])

5. Os colaboradores só conseguem participar em reuniões em slots que têm disponibilidade$$\forall_{t<T}\forall_{c<C}\forall_{p<P}\forall_{s<S},disp[c][t] = 0 \rightarrow X_{s,t,p,c} = 0 $$

In [46]:
for t in range(T):
    for c in range(C):
        for p in range(P):
            for s in range(S):
                if disp[c][t] == 0:
                    horario.Add(X[s,t,p,c] == 0)

6. Os colaboradores não podem ter mais do que uma reunião no mesmo slot$$\forall_{t<T}\forall_{c<C}\forall_{s<S},\sum_{p<P} X_{s,t,p,c} \le 1$$

In [47]:
for t in range(T):
    for c in range(C):
        for s in range(S):
            horario.Add(sum([X[s,t,p,c] for p in range(P)]) <= 1)

 7. Cada colaborador só pode participar em reuniões de projetos em que está incluido$$ \forall_{c<C}\forall_{p<P}\forall_{t<T}\forall_{s<S}, c \notin projetos[p][0] \rightarrow X_{s,t,p,c} = 0 $$

In [48]:
for t in range(T):
    for c in range(C):
        for p in range(P):
            for s in range(S):
                if c not in projetos[p][0]:
                    horario.Add(X[s,t,p,c] == 0)

8.Cada projeto só pode ter 1 ou 0 reuniões em cada slot $$\forall_{t<T}\forall_{p<P},\sum_{s<S}X_{s,t,p,projetos[p][1]} \le 1$$

In [49]:
for t in range(T):
    for p in range(P):
        horario.Add(sum([X[s,t,p,projetos[p][1]] for s in range(S)]) <= 1)

1. Maximizar o número de reuniões efetivamente realizadas

In [50]:
horario.Maximize(sum([X[s,t,p,projetos[p][1]] for s in range(S) for t in range(T) for p in range(P)]))

2. Minimizar o número médio de reuniões por participante

In [51]:
for s in range(S):
    for p in range(P):
        horario.Minimize(sum([X[s,t,p,c] for c in range(C) for t in range(T)])/C)  

## Imprimir o horário:

In [ ]:
status = horario.Solve()
if status == pywraplp.Solver.OPTIMAL:
    for t in range(T):
        print("\nSlot " + str(t) + ":")
        for s in range(S):
            print("----------------------------\n\tSala " + str(s) + ":")
            for p in range(P):
                colabs = str()
                print("\n\t\tProjeto " + str(p) + ":")
            
                for c in range(C):
                    
                    if (int(X[s,t,p,c].solution_value())) != 0:
                        colabs = colabs + " " + str(int(X[s,t,p,c].solution_value()) * c)
                print("\t\t" + colabs)        
        print(".......................................................")
            

else:
    print("Não há solução")